In [211]:
# !pip install numpy #==1.19.5
# !pip install torch #==1.7.1  
# !pip install --upgrade pip setuptools
# !pip install transformers
# !pip install sklearn #==0.0

# !pip install pandas
# !pip install openpyxl
# !pip install pyarrow
# !pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.3/685.3 kB 9.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [180]:
import pandas as pd

In [4]:
import torch


In [5]:
from transformers import BertTokenizer, BertModel#, AdamW

/Users/abhi/Desktop/Chaudhury_Lab/QuoteR_Imp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/abhi/Desktop/Chaudhury_Lab/QuoteR_Imp/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from torch.optim import AdamW

In [3]:
# from transformers import BertSememeModel

In [7]:
from transformers.models.bert import BertPreTrainedModel
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.models.bert.modeling_bert import BERT_INPUTS_DOCSTRING, _TOKENIZER_FOR_DOC, BertSememeEmbeddings, BertEncoder, BertPooler
# TODO: add BertSemeModel
class BertSememeModel(BertPreTrainedModel):
    """

    The model can behave as an encoder (with only self-attention) as well
    as a decoder, in which case a layer of cross-attention is added between
    the self-attention layers, following the architecture described in `Attention is all you need`_ by Ashish Vaswani,
    Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.

    To behave as an decoder the model needs to be initialized with the
    :obj:`is_decoder` argument of the configuration set to :obj:`True`; an
    :obj:`encoder_hidden_states` is expected as an input to the forward pass.

    .. _`Attention is all you need`:
        https://arxiv.org/abs/1706.03762

    """

    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.embeddings = BertSememeEmbeddings(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)

        self.init_weights()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """ Prunes heads of the model.
            heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
            See base class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    # # @add_start_docstrings_to_callable
    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("(batch_size, sequence_length)"))
    @add_code_sample_docstrings(processor_class=_TOKENIZER_FOR_DOC, checkpoint="bert-base-uncased")
    def forward(
        self,
        quote_ids,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        r"""
    Return:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        last_hidden_state (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.
        pooler_output (:obj:`torch.FloatTensor`: of shape :obj:`(batch_size, hidden_size)`):
            Last layer hidden-state of the first token of the sequence (classification token)
            further processed by a Linear layer and a Tanh activation function. The Linear
            layer weights are trained from the next sentence prediction (classification)
            objective during pre-training.

            This output is usually *not* a good summary
            of the semantic content of the input, you're often better with averaging or pooling
            the sequence of hidden-states for the whole input sequence.
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``output_hidden_states=True`` is passed or when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``output_attentions=True`` is passed or when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if attention_mask is None:
            attention_mask = torch.ones(input_shape, device=device)
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D ou 3D attention mask is provided for the cross-attention
        # we need to make broadcastabe to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            quote_ids=quote_ids, input_ids=input_ids, position_ids=position_ids, token_type_ids=token_type_ids, inputs_embeds=inputs_embeds
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output)

        outputs = (sequence_output, pooled_output,) + encoder_outputs[
            1:
        ]  # add hidden_states and attentions if they are here
        return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)

In [8]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import time
import random
from sklearn.metrics import accuracy_score
import numpy as np

In [9]:

random.seed(42)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch = 4
sample_num = 19
learning_rate = 3e-5

In [10]:

# loading dataset
def load_data(path):
    # return words list and labels
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [line.strip().lower().split('\t') for line in lines]
        train_former = [line[0] for line in lines[:101171]]
        train_quote = [line[1] for line in lines[:101171]]
        train_latter = [line[2] for line in lines[:101171]]
        valid_former = [line[0] for line in lines[101171:113942]]
        valid_quote = [line[1] for line in lines[101171:113942]]
        valid_latter = [line[2] for line in lines[101171:113942]]
        test_former = [line[0] for line in lines[113942:]]
        test_quote = [line[1] for line in lines[113942:]]
        test_latter = [line[2] for line in lines[113942:]]
        all_quotes = train_quote + valid_quote + test_quote
    # 去重
    all_quotes = list(set(all_quotes))
    all_quotes.sort()
    y_train = [all_quotes.index(q) for q in train_quote]
    y_valid = [all_quotes.index(q) for q in valid_quote]
    y_test = [all_quotes.index(q) for q in test_quote]

    return train_former, train_latter, train_quote, valid_former, valid_latter, valid_quote, test_former, test_latter, test_quote, torch.LongTensor(
        y_train), torch.LongTensor(y_valid), torch.LongTensor(
            y_test), all_quotes


In [11]:

print("loading dataset......")
data_path = "./data/english.txt"
train_former, train_latter, train_quote, valid_former, valid_latter, valid_quote, test_former, test_latter, test_quote, y_train, y_valid, y_test, all_quotes = load_data(
    data_path)
print("train  valid  test:", len(train_former), len(valid_former), len(test_former))
print("all quotes: ", len(all_quotes))
print("train quote:", len(list(set(train_quote))))
print("valid quote:", len(list(set(valid_quote))))
print("test quote:", len(list(set(test_quote))))

# get the Tokenizer used for pretraining model
PRETRAINED_MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


loading dataset......
train  valid  test: 101171 12771 12771
all quotes:  6108
train quote: 6008
valid quote: 6108
test quote: 6108


tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 19.8kB/s]


In [12]:
# how many total contexts?
print(f"{101171 + 12771 + 12771} total contexts in dset")

126713 total contexts in dset


In [13]:
print(train_former[1])
print(train_quote[1])
print(train_latter[1])

to be done? is the first question. how is it to be done is a question which is secondary and its discussion is useless until the first is settled. too much state drove ginx's baby into the thames. what's 
everybody's business is nobody's business.
 if the uncountable babies of innumerable ginx's are to be aided, some one must aid them for the mere pleasure there is in loving-kindness. a baby is a human being, not a problem. a baby can't be explained away


In [14]:
def make_context_tensors(former, latter):
    input_ids = []
    token_type_ids = []
    attention_masks = []
    mask_ids = []
    for f, l in zip(former, latter):
        sent = f + "[MASK]" + l
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=150,
                                             padding='max_length',  # instead of pad_to_max_length=True
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
        token_type_ids.append(encoded_dict['token_type_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        mask_index = encoded_dict['input_ids'][0].tolist().index(103)
        mask_ids.append(mask_index)
    input_ids = torch.cat(input_ids, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, token_type_ids, attention_masks, torch.LongTensor(
        mask_ids)



In [15]:

print("loading train and valid data......")
train_input_ids, train_token_type_ids, train_attention_masks, train_mask_ids = make_context_tensors(
    train_former, train_latter)
valid_input_ids, valid_token_type_ids, valid_attention_masks, valid_mask_ids = make_context_tensors(
    valid_former, valid_latter)
print("train bert input:")
print(train_input_ids.shape, train_token_type_ids.shape,
      train_attention_masks.shape, train_mask_ids.shape)
print("valid bert input:")
print(valid_input_ids.shape, valid_token_type_ids.shape,
      valid_attention_masks.shape, valid_mask_ids.shape)


loading train and valid data......
train bert input:
torch.Size([101171, 150]) torch.Size([101171, 150]) torch.Size([101171, 150]) torch.Size([101171])
valid bert input:
torch.Size([12771, 150]) torch.Size([12771, 150]) torch.Size([12771, 150]) torch.Size([12771])


In [16]:

# Dataset and DataLoader
class Dataset(Dataset):
    def __init__(self, input_ids, token_type_ids, attention_masks, mask_ids,
                 quote):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_masks = attention_masks
        self.mask_ids = mask_ids
        self.quote = quote

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.quote is None:
            return self.input_ids[idx], self.token_type_ids[
                idx], self.attention_masks[idx], self.mask_ids[idx]
        return self.input_ids[idx], self.token_type_ids[
            idx], self.attention_masks[idx], self.mask_ids[idx], self.quote[
                idx]


print("loading train and valid dataloader ...")
train_dataset = Dataset(input_ids=train_input_ids,
                        token_type_ids=train_token_type_ids,
                        attention_masks=train_attention_masks,
                        mask_ids=train_mask_ids,
                        quote=train_quote)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=0) # 2)
valid_dataset = Dataset(input_ids=valid_input_ids,
                        token_type_ids=valid_token_type_ids,
                        attention_masks=valid_attention_masks,
                        mask_ids=valid_mask_ids,
                        quote=valid_quote)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch,
                          shuffle=True,
                          num_workers=0) # 2)


#  generat negative examples according to num
def generate_quotes(quote, num):
    quotes_selcet = all_quotes[:]
    quotes_selcet.remove(quote)
    quotes = random.sample(quotes_selcet, num)
    quotes.append(quote)
    random.shuffle(quotes)
    return quotes


def make_quote_tensors(quote):
    quotes = generate_quotes(quote, num=sample_num)
    label = quotes.index(quote)
    input_ids = []
    for q in quotes:
        encoded_dict = tokenizer.encode_plus(q,
                                             add_special_tokens=True,
                                             max_length=80,
                                             padding='max_length',  # instead of pad_to_max_length=True
                                             truncation=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])
    input_ids = torch.cat(input_ids, 0)  # [num, 80]
    quote_ids = [all_quotes.index(q) for q in quotes]
    return input_ids, label, torch.LongTensor(quote_ids)



loading train and valid dataloader ...


In [17]:

# Define network
class Context_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.5)

    def forward(self, context_input_ids, context_token_type_ids,
                context_attention_masks, mask_ids):
        outputs = self.bert_model(input_ids=context_input_ids,
                                  token_type_ids=context_token_type_ids,
                                  attention_mask=context_attention_masks)
        last_hidden_state = outputs[0]  # [batch_size, sequence_length, hidden_size]
        all_context = []
        for i in range(len(last_hidden_state)):
            hidden_state = last_hidden_state[i]  # [sequence_length, hidden_size]
            mask = hidden_state[mask_ids[i]]
            mask = self.dropout(mask)
            context = mask.unsqueeze(dim=0)  # context: [1, hidden_size]
            all_context.append(context)
        all_context = torch.cat(all_context, dim=0)  # all_context: [batch, hidden_size]
        return all_context


class Quote_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertSememeModel.from_pretrained(PRETRAINED_MODEL_NAME)
        # self.dropout = nn.Dropout(0.5)

    def forward(self, quotes):
        quote_tensor = []
        labels = []
        for quote in quotes:
            quote_input_ids, label, quote_ids = make_quote_tensors(quote)
            quote_input_ids = quote_input_ids.to(device)
            quote_ids = quote_ids.to(device)
            outputs = self.bert_model(input_ids=quote_input_ids, quote_ids=quote_ids)
            last_hidden_state = outputs[0]  # (num, sequence_length, hidden_size))
            # last_hidden_state = self.dropout(last_hidden_state)
            output = torch.mean(last_hidden_state, dim=1)  # (num, hidden_size))
            quote_tensor.append(output)
            labels.append(label)
        quote_tensor = torch.stack(quote_tensor, dim=0)  # (batch, num, hidden_size))
        return quote_tensor, labels


In [18]:

class QuotRec_Net(nn.Module):
    def __init__(self, contex_model, quote_model):
        super().__init__()
        self.contex_model = contex_model
        self.quote_model = quote_model

    def forward(self, input_ids, token_type_ids, attention_masks, mask_ids,
                quotes):
        # context_output: [batch, hidden_size]
        context_output = self.contex_model(input_ids, token_type_ids,
                                           attention_masks, mask_ids)
        context_output = context_output.unsqueeze(dim=1)  # [batch, 1, hidden_size]
        # quote_output: [batch, num, hidden_size]  labels: [batch]
        quote_output, labels = self.quote_model(quotes)
        quote_output = quote_output.permute(0, 2, 1)
        outputs = torch.matmul(context_output, quote_output).squeeze(
            dim=1)  # output: [batch, num_quotes]
        return outputs, torch.LongTensor(labels)


In [19]:

print("loading model......")
contex_model = Context_Encoder()
quote_model = Quote_Encoder()
model = QuotRec_Net(contex_model, quote_model)
model.to(device)


loading model......


Some weights of BertSememeModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.sememe_embeddings.lut.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


QuotRec_Net(
  (contex_model): Context_Encoder(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [20]:

def training(model, epoch, train, valid, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(
        total, trainable))
    t_batch = len(train)
    v_batch = len(valid)
    # NOTE: the lengths of the datasets are too long
    # in training takes ~11 sec for each data point
    # say i want max 0.5 hr
    max_num_data = int(0.5*60*60/11)
    print(f"max # data: {max_num_data}")
    if t_batch > max_num_data:
        # train = train[0:max_num_data]
        t_batch = max_num_data
    
    if v_batch > max_num_data:
        # valid = valid[0:max_num_data]
        v_batch = max_num_data

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_acc = 0
    count = 0
    for epoch in range(epoch):
        start = time.perf_counter()
        total_loss, total_acc = 0, 0
        print("epoch: ", epoch + 1)
        # train
        for i, (input_ids, token_type_ids, attention_masks, mask_ids,
                quotes) in enumerate(train):
            if i >= t_batch:
                break

            print(f"On data {i}/{len(train)}")
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            optimizer.zero_grad()
            outputs, labels = model(input_ids, token_type_ids, attention_masks,
                                    mask_ids, quotes)
            labels = labels.to(device, dtype=torch.long)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(outputs.cpu().data, 1)
            acc = accuracy_score(pred, labels.cpu())
            total_loss += loss.item()
            total_acc += acc
        print('Train | Loss:{:.5f} Acc:{:.3f}'.format(total_loss,
                                                      total_acc / t_batch))
        # validation
        model.eval()
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (input_ids, token_type_ids, attention_masks, mask_ids,
                    quotes) in enumerate(valid):
                if i >= v_batch:
                    break
                
                print(f"On data {i}/{len(valid)}")

                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                mask_ids = mask_ids.to(device, dtype=torch.long)
                outputs, labels = model(input_ids, token_type_ids,
                                        attention_masks, mask_ids, quotes)
                labels = labels.to(device, dtype=torch.long)
                loss = criterion(outputs, labels)
                _, pred = torch.max(outputs.cpu().data, 1)
                acc = accuracy_score(pred, labels.cpu())
                total_loss += loss.item()
                total_acc += acc
            print('Valid | Loss:{:.5f} Acc:{:.3f}'.format(
                total_loss, total_acc / v_batch))
            if total_acc > best_acc:
                best_acc = total_acc
                if os.path.exists("./model"):
                    pass
                else:
                    os.mkdir("./model")
                torch.save(
                    model.quote_model.state_dict(), "./model/english_quote.pth")
                torch.save(
                    model.contex_model.state_dict(), "./model/english_context.pth")
                print('saving model with Acc {:.3f} '.format(total_acc / v_batch))
                count = 0
            else:
                count += 1
        model.train()
        end = time.perf_counter()
        print('epoch running time:{:.0f}s'.format(end - start))
        # early stopping
        if count == 3:
            break


In [40]:
import warnings
warnings.filterwarnings("ignore")

training(model=model,
         epoch= 4, # TODO: # 40,
         train=train_loader,
         valid=valid_loader,
         device=device)

warnings.resetwarnings()


start training, parameter total:220290048, trainable:220290048

max # data: 163
epoch:  1
On data 0/25293
On data 1/25293
On data 2/25293
On data 3/25293
On data 4/25293
On data 5/25293
On data 6/25293
On data 7/25293
On data 8/25293
On data 9/25293
On data 10/25293
On data 11/25293
On data 12/25293
On data 13/25293
On data 14/25293
On data 15/25293
On data 16/25293
On data 17/25293
On data 18/25293
On data 19/25293
On data 20/25293
On data 21/25293
On data 22/25293
On data 23/25293
On data 24/25293
On data 25/25293
On data 26/25293
On data 27/25293
On data 28/25293
On data 29/25293
On data 30/25293
On data 31/25293
On data 32/25293
On data 33/25293
On data 34/25293
On data 35/25293
On data 36/25293
On data 37/25293
On data 38/25293
On data 39/25293
On data 40/25293
On data 41/25293
On data 42/25293
On data 43/25293
On data 44/25293
On data 45/25293
On data 46/25293
On data 47/25293
On data 48/25293
On data 49/25293
On data 50/25293
On data 51/25293
On data 52/25293
On data 53/25293
O

In [21]:
# TODO: figure out attention mask stuff
q = all_quotes[0]
encoded_dict = tokenizer.encode_plus(q,
                                    add_special_tokens=True,
                                    max_length=80,
                                    padding='max_length',  # instead of pad_to_max_length=True
                                    truncation=True,
                                    return_tensors='pt',
                                    return_attention_mask=True)

In [22]:
encoded_dict['input_ids']

tensor([[  101,  1005,  2498, 16958,  2004,  2204,  2004, 15629,  5683,  1012,
          1005,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [23]:
encoded_dict['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [24]:

# make quotes to bert tensor
def make_tensors(quotes):
    input_ids = []
    attention_masks = []
    for q in quotes:
        encoded_dict = tokenizer.encode_plus(q,
                                             add_special_tokens=True,
                                             max_length=80,
                                             padding='max_length',  # instead of pad_to_max_length=True
                                             truncation=True,
                                             return_tensors='pt',
                                             return_attention_mask=True)
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, 0)
    attention_masks = torch.cat(attention_masks, 0)
    return input_ids, attention_masks

In [25]:

quote_input_ids, quote_attention_masks = make_tensors(all_quotes)
print("quote bert input:")
print(quote_input_ids.shape)


quote bert input:
torch.Size([6108, 80])


In [26]:
quote_attention_masks[0].unsqueeze(dim=0)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [27]:

# Generate sentence vector for quotes
quote_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
model_dict = quote_model.state_dict()
save_model_state = torch.load("./model/english_quote.pth")
state_dict = {k[11:]: v for k, v in save_model_state.items() if k[11:] in model_dict.keys()}
model_dict.update(state_dict)
quote_model.load_state_dict(model_dict)
quote_model = quote_model.to(device)
quote_input_ids = quote_input_ids.to(device)

In [28]:

quote_embeddings = []
quote_model.eval()
with torch.no_grad():
    i = 0
    for input_ids in quote_input_ids:
        i += 1
        input_ids = input_ids.unsqueeze(dim=0)
        # TODO: check if passing in attention mask was the right move
        attention_mask = quote_attention_masks[i-1].unsqueeze(dim=0)
        
        outputs = quote_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs[0]  # hidden_states:[batch_size, sequence_length, hidden_size]
        quote_tensor = torch.mean(hidden_states, dim=1)  # quote_tensor: [batch_size, hidden_size]
        quote_embeddings.append(quote_tensor)
    quote_embeddings = torch.cat(quote_embeddings, dim=0)
print("quote tensor:")
print(quote_embeddings.shape)

quote tensor:
torch.Size([6108, 768])


In [29]:

# Use the mask method for training
class QuotRecNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids, token_type_ids, attention_masks,
                mask_ids, quote_tensor):
        outputs = self.bert_model(input_ids=input_ids,
                                  token_type_ids=token_type_ids,
                                  attention_mask=attention_masks)
        last_hidden_state = outputs[0]  # [batch_size, sequence_length, hidden_size]
        all_outputs = []
        for i in range(len(last_hidden_state)):
            hidden_state = last_hidden_state[i]  # [sequence_length, hidden_size]
            mask = hidden_state[mask_ids[i]]
            context = self.dropout(mask)
            context = context.unsqueeze(dim=0)  # context: [1, hidden_size]
            # quote_tensor: [num_class, hidden_size]
            output = torch.mm(context, quote_tensor.t())  # outputs: [1, num_class]
            all_outputs.append(output)
        all_outputs = torch.cat(all_outputs, dim=0)  # all_outputs: [batch, num_class]
        return all_outputs


In [30]:

print("loading model......")
model = QuotRecNet()
model.to(device)


loading model......


QuotRecNet(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [31]:

# get rank
def rank_gold(predicts, golds):
    ranks = []
    ps = predicts.data.cpu().numpy()
    gs = golds.cpu().numpy()
    for i in range(len(ps)):
        predict = ps[i]
        gold_index = gs[i]
        predict_value = predict[gold_index]
        predict_sort = sorted(predict, reverse=True)
        predict_index = predict_sort.index(predict_value)
        if predict_index == -1:
            break
        ranks.append(predict_index)
    return ranks


In [32]:

# get NDCG@5
def get_NDCG(ranks):
    total = 0.0
    for r in ranks:
        if r < 5:  # k=5
            total += 1.0 / np.log2(r + 2)
    return total / len(ranks)



In [33]:

# get recall@k
def recall(predicts, golds):
    predicts = predicts.data.cpu().numpy()
    golds = golds.cpu().numpy()
    predicts_index = np.argsort(-predicts, axis=1)
    recall_1, recall_3, recall_5, recall_10, recall_20, recall_30 = 0, 0, 0, 0, 0, 0
    recall_100, recall_300, recall_500, recall_1000 = 0, 0, 0, 0
    for i in range(len(golds)):
        if golds[i] in predicts_index[i][:1000]:
            recall_1000 += 1
            if golds[i] in predicts_index[i][:500]:
                recall_500 += 1
                if golds[i] in predicts_index[i][:300]:
                    recall_300 += 1
                    if golds[i] in predicts_index[i][:100]:
                        recall_100 += 1
                        if golds[i] in predicts_index[i][:30]:
                            recall_30 += 1
                            if golds[i] in predicts_index[i][:20]:
                                recall_20 += 1
                                if golds[i] in predicts_index[i][:10]:
                                    recall_10 += 1
                                    if golds[i] in predicts_index[i][:5]:
                                        recall_5 += 1
                                        if golds[i] in predicts_index[i][:3]:
                                            recall_3 += 1
                                            if golds[i] in predicts_index[
                                                    i][:1]:
                                                recall_1 += 1
    return recall_1, recall_3, recall_5, recall_10, recall_20, recall_30, recall_100, recall_300, recall_500, recall_1000


In [34]:

def training_mask(model, epoch, train, valid, quote_tensor, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(
        total, trainable))
    t_batch = len(train)
    v_batch = len(valid)

    # NOTE: the lengths of the datasets are too long
    # in training takes ~14 sec for each data point
    # say i want max 0.5 hr
    max_num_data = int(0.5*60*60/14)
    print(f"max # data: {max_num_data}")
    if t_batch > max_num_data:
        # train = train[0:max_num_data]
        t_batch = max_num_data
    
    if v_batch > max_num_data:
        # valid = valid[0:max_num_data]
        v_batch = max_num_data

    learning_rate = 5e-5
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_MRR = 0
    count = 0
    quote_tensor = quote_tensor.to(device)
    for epoch in range(epoch):
        start = time.perf_counter()
        print("epoch: ", epoch + 1)
        total_loss, total_MRR, total_NDCG = 0, 0, 0
        # train
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(train):
            if i >= t_batch:
                break

            print(f"On data {i}/{len(train)}")

            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.long)
            optimizer.zero_grad()
            outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)  # outputs: (batch, num_class)
            # print("outputs:", outputs.shape)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            ranks = rank_gold(outputs, labels)
            MRR = np.average([1.0 / (r + 1) for r in ranks])
            NDCG = get_NDCG(ranks)
            total_loss += loss.item()
            total_MRR += MRR
            total_NDCG += NDCG
        end = time.perf_counter()
        print('Epoch running time :{:.0f}'.format(end - start))
        print('Train | Loss:{:.3f} MRR: {:.3f} NDCG: {:.3f}'.format(total_loss, total_MRR/t_batch, total_NDCG/t_batch))

        # validation
        model.eval()
        with torch.no_grad():
            total_loss, total_MRR, total_NDCG = 0, 0, 0
            for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(valid):
                if i >= v_batch:
                    break

                print(f"On data {i}/{len(valid)}")

                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                mask_ids = mask_ids.to(device, dtype=torch.long)
                labels = labels.to(device, dtype=torch.long)
                outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
                loss = criterion(outputs, labels)
                ranks = rank_gold(outputs, labels)
                MRR = np.average([1.0 / (r + 1) for r in ranks])
                NDCG = get_NDCG(ranks)
                total_loss += loss.item()
                total_MRR += MRR
                total_NDCG += NDCG
            print("Valid | Loss:{:.5f} MRR: {:.3f} NDCG: {:.3f}".format(
                total_loss, total_MRR / v_batch,
                total_NDCG / v_batch))
        if total_MRR > best_MRR:
            best_MRR = total_MRR
            torch.save(model, "./model/model_english.model")
            print('saving model with MRR {:.3f} NDCG: {:.3f}'.format(
                total_MRR / v_batch, total_NDCG / v_batch))
            count = 0
        else:
            learning_rate = learning_rate * 0.9
            count += 1
        # early stopping
        if count == 3:
            break
        model.train()


In [35]:

# Mask Dataset and DataLoader
class Dataset_Mask(Dataset):

    def __init__(self, input_ids, token_type_ids, attention_masks, mask_ids,
                 y):
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_masks = attention_masks
        self.mask_ids = mask_ids
        self.label = y

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.label is None:
            return self.input_ids[idx], self.token_type_ids[
                idx], self.attention_masks[idx], self.mask_ids[idx]
        return self.input_ids[idx], self.token_type_ids[
            idx], self.attention_masks[idx], self.mask_ids[idx], self.label[
                idx]


In [36]:

print("loading train and valid dataloader ...")
train_dataset_mask = Dataset_Mask(input_ids=train_input_ids,
                                  token_type_ids=train_token_type_ids,
                                  attention_masks=train_attention_masks,
                                  mask_ids=train_mask_ids,
                                  y=y_train)
train_loader_mask = DataLoader(dataset=train_dataset_mask,
                               batch_size=64,
                               shuffle=True,
                               num_workers= 0) # 2)
valid_dataset_mask = Dataset_Mask(input_ids=valid_input_ids,
                                  token_type_ids=valid_token_type_ids,
                                  attention_masks=valid_attention_masks,
                                  mask_ids=valid_mask_ids,
                                  y=y_valid)
valid_loader_mask = DataLoader(dataset=valid_dataset_mask,
                               batch_size=64,
                               shuffle=True,
                               num_workers=0)# 2)


loading train and valid dataloader ...


In [84]:
print("start training......")
training_mask(model=model,
              epoch=4, #TODO: 40,
              train=train_loader_mask,
              valid=valid_loader_mask,
              quote_tensor=quote_embeddings,
              device=device)

start traing......

start training, parameter total:109482240, trainable:109482240

max # data: 128
epoch:  1
On data 0/1581
On data 1/1581
On data 2/1581
On data 3/1581
On data 4/1581
On data 5/1581
On data 6/1581
On data 7/1581
On data 8/1581
On data 9/1581
On data 10/1581
On data 11/1581
On data 12/1581
On data 13/1581
On data 14/1581
On data 15/1581
On data 16/1581
On data 17/1581
On data 18/1581
On data 19/1581
On data 20/1581
On data 21/1581
On data 22/1581
On data 23/1581
On data 24/1581
On data 25/1581
On data 26/1581
On data 27/1581
On data 28/1581
On data 29/1581
On data 30/1581
On data 31/1581
On data 32/1581
On data 33/1581
On data 34/1581
On data 35/1581
On data 36/1581
On data 37/1581
On data 38/1581
On data 39/1581
On data 40/1581
On data 41/1581
On data 42/1581
On data 43/1581
On data 44/1581
On data 45/1581
On data 46/1581
On data 47/1581
On data 48/1581
On data 49/1581
On data 50/1581
On data 51/1581
On data 52/1581
On data 53/1581
On data 54/1581
On data 55/1581
On d

In [37]:
def test(model, test_loader, quote_tensor, device):
    print("start test......")
    model.eval()
    t_batch = len(test_loader)
    criterion = nn.CrossEntropyLoss()
    quote_tensor = quote_tensor.to(device)
    with torch.no_grad():
        total_loss, total_MRR, total_NDCG, total_ranks = 0, 0, 0, 0
        total_recall_1, total_recall_3, total_recall_5, total_recall_10, total_recall_20, total_recall_30 = 0, 0, 0, 0, 0, 0
        total_recall_100, total_recall_300, total_recall_500, total_recall_1000 = 0, 0, 0, 0
        all_ranks = []
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(test_loader):

            print(f"On data {i}/{len(test_loader)}")

            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
            loss = criterion(outputs, labels)
            ranks = rank_gold(outputs, labels)
            all_ranks.extend(ranks)
            MRR = np.average([1.0 / (r + 1) for r in ranks])
            NDCG = get_NDCG(ranks)
            recall_1, recall_3, recall_5, recall_10, recall_20, recall_30, recall_100, recall_300, recall_500, recall_1000 = recall(
                outputs, labels)
            total_loss += loss.item()
            total_MRR += MRR
            total_NDCG += NDCG
            total_ranks += np.sum(ranks)
            total_recall_1 += recall_1
            total_recall_3 += recall_3
            total_recall_5 += recall_5
            total_recall_10 += recall_10
            total_recall_20 += recall_20
            total_recall_30 += recall_30
            total_recall_100 += recall_100
            total_recall_300 += recall_300
            total_recall_500 += recall_500
            total_recall_1000 += recall_1000
        print(len(all_ranks))
        print(
            "Test | Loss:{:.5f} MRR: {:.3f} NDCG: {:.3f} Mean Rank: {:.0f} Median Rank: {:.0f} Variance: {:.0f}"
            .format(total_loss, total_MRR / t_batch,
                    total_NDCG / t_batch, np.mean(all_ranks),
                    np.median(all_ranks)+1,
                    np.std(all_ranks)))
        print(
            "Recall@1:{:.4f} Recall@3: {:.4f} Recall@5: {:.4f} Recall@10: {:.4f} Recall@20: {:.4f} Recall@30: {:.4f} Recall@100: {:.4f} Recall@300: {:.4f} Recall@500: {:.4f} Recall@1000: {:.4f}"
            .format(
                total_recall_1 / len(y_test), total_recall_3 / len(y_test),
                total_recall_5 / len(y_test), total_recall_10 / len(y_test),
                total_recall_20 / len(y_test), total_recall_30 / len(y_test),
                total_recall_100 / len(y_test), total_recall_300 / len(y_test),
                total_recall_500 / len(y_test),
                total_recall_1000 / len(y_test)))


In [38]:
print(test_former[0])
print(test_latter[0])

archived video of that message here: biblical evidence for natural law listen to the scripture: "for when the gentiles, which have not the law, do by nature the things contained in the law, these, having not the law, are 
 which shew the work of the law written in their hearts, their conscience also bearing witness, and their thoughts the mean while accusing or else excusing one another." the great theologians and bible scholars of yesteryear all understood the


In [39]:

print("loading test tensor......")
test_input_ids, test_token_type_ids, test_attention_masks, test_mask_ids = make_context_tensors(test_former, test_latter)
test_dataset_mask = Dataset_Mask(input_ids=test_input_ids,
                                 token_type_ids=test_token_type_ids,
                                 attention_masks=test_attention_masks,
                                 mask_ids=test_mask_ids,
                                 y=y_test)
test_loader_mask = DataLoader(dataset=test_dataset_mask,
                              batch_size=64,
                              num_workers=0) #TODO: 2)

loading test tensor......


In [40]:
# decode what the test input is
decoded_sentence = tokenizer.decode(test_input_ids[0])
print(decoded_sentence)

[CLS] archived video of that message here : biblical evidence for natural law listen to the scripture : " for when the gentiles, which have not the law, do by nature the things contained in the law, these, having not the law, are [MASK] which shew the work of the law written in their hearts, their conscience also bearing witness, and their thoughts the mean while accusing or else excusing one another. " the great theologians and bible scholars of yesteryear all understood the [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [ ]:
# ~ 6 seconds for each test --> 20 minutes run time :(
print('loading model ...')
model = torch.load('./model/model_english.model')
model.to(device)
test(model=model,
     test_loader=test_loader_mask,
     quote_tensor=quote_embeddings,
     device=device)

Readable testing of what the model does

In [166]:
TEST_BATCH_SIZE = 2

test_loader_mask_0 = DataLoader(dataset=test_dataset_mask,
                              batch_size=TEST_BATCH_SIZE,
                              num_workers=0)

In [123]:
# DEPRECATED:

# only test the 0th one
def OLD_test_readable_0(model, test_loader, quote_tensor, device):
    print("start test......")
    model.eval()
    t_batch = len(test_loader)
    criterion = nn.CrossEntropyLoss()
    quote_tensor = quote_tensor.to(device)
    with torch.no_grad():
        total_loss, total_MRR, total_NDCG, total_ranks = 0, 0, 0, 0
        total_recall_1, total_recall_3, total_recall_5, total_recall_10, total_recall_20, total_recall_30 = 0, 0, 0, 0, 0, 0
        total_recall_100, total_recall_300, total_recall_500, total_recall_1000 = 0, 0, 0, 0
        all_ranks = []
        for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(test_loader):
            if i > 0:
                break

            print(f"On data {i}/{len(test_loader)}")

            # decode what the test input is
            # 64 things in the batch
            print(f"input shape: {input_ids.shape}")
            # Decode the input_ids of sentence 0
            # this is decoding the sentence tokens --> words
            decoded_sentence = tokenizer.decode(input_ids[0], attention_mask=attention_masks[0])
            print("input0:", decoded_sentence)

            input_ids = input_ids.to(device)

            token_type_ids = token_type_ids.to(device)
            attention_masks = attention_masks.to(device)
            mask_ids = mask_ids.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
            
            print(f"outputs.shape: {outputs.shape}")
            # Print the list of NL sentences arranged by rank for input_ids[0]
            print(f"labels: {labels}")
            print(f"outputs for input0: {outputs}")
            print("Predicted Sentences for input0:")
            
            # TODO: figure this out
            ps = outputs.data.cpu().numpy()
            print(f"len(ps): {len(ps)}")
            gs = labels.cpu().numpy()
            i=0 # for i in range(len(ps)):
            predict = ps[i] # predictions corresponding to input 0??
            gold_index = gs[i] # 
            print(f"predict: {predict}\ngold: {gold_index}")
            predict_value = predict[gold_index]
            predict_sort = sorted(predict, reverse=True)
            predict_index = predict_sort.index(predict_value)
            if predict_index == -1:
                break

        #     loss = criterion(outputs, labels)
        #     ranks = rank_gold(outputs, labels)

        #     print(f"ranks = {ranks}")

        #     all_ranks.extend(ranks)
        #     MRR = np.average([1.0 / (r + 1) for r in ranks])
        #     NDCG = get_NDCG(ranks)
        #     recall_1, recall_3, recall_5, recall_10, recall_20, recall_30, recall_100, recall_300, recall_500, recall_1000 = recall(
        #         outputs, labels)
        #     total_loss += loss.item()
        #     total_MRR += MRR
        #     total_NDCG += NDCG
        #     total_ranks += np.sum(ranks)
        #     total_recall_1 += recall_1
        #     total_recall_3 += recall_3
        #     total_recall_5 += recall_5
        #     total_recall_10 += recall_10
        #     total_recall_20 += recall_20
        #     total_recall_30 += recall_30
        #     total_recall_100 += recall_100
        #     total_recall_300 += recall_300
        #     total_recall_500 += recall_500
        #     total_recall_1000 += recall_1000
        # print(f"len(all_ranks) = {len(all_ranks)}")
        # print(
        #     "Test | Loss:{:.5f} MRR: {:.3f} NDCG: {:.3f} Mean Rank: {:.0f} Median Rank: {:.0f} Variance: {:.0f}"
        #     .format(total_loss, total_MRR / t_batch,
        #             total_NDCG / t_batch, np.mean(all_ranks),
        #             np.median(all_ranks)+1,
        #             np.std(all_ranks)))
        # print(
        #     "Recall@1:{:.4f} Recall@3: {:.4f} Recall@5: {:.4f} Recall@10: {:.4f} Recall@20: {:.4f} Recall@30: {:.4f} Recall@100: {:.4f} Recall@300: {:.4f} Recall@500: {:.4f} Recall@1000: {:.4f}"
        #     .format(
        #         total_recall_1 / len(y_test), total_recall_3 / len(y_test),
        #         total_recall_5 / len(y_test), total_recall_10 / len(y_test),
        #         total_recall_20 / len(y_test), total_recall_30 / len(y_test),
        #         total_recall_100 / len(y_test), total_recall_300 / len(y_test),
        #         total_recall_500 / len(y_test),
        #         total_recall_1000 / len(y_test)))


In [124]:
test_readable_0(model=model,
     test_loader=test_loader_mask_0,
     quote_tensor=quote_embeddings,
     device=device)

start test......
On data 0/6386
input shape: torch.Size([2, 150])
input0: [CLS] archived video of that message here : biblical evidence for natural law listen to the scripture : " for when the gentiles, which have not the law, do by nature the things contained in the law, these, having not the law, are [MASK] which shew the work of the law written in their hearts, their conscience also bearing witness, and their thoughts the mean while accusing or else excusing one another. " the great theologians and bible scholars of yesteryear all understood the [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
outputs.shape: torch.Size([2, 6108])
labels: tensor([ 76, 867])
outputs for input0: tensor([[38.2892, 38.7050, 37.6607,  ..., 39.3191, 31.3

In [197]:
model = model
test_loader = test_loader_mask
quote_tensor = quote_embeddings
device = device

In [199]:
len(y_test)

12771

In [200]:
print("start test......")
model.eval()
t_batch = len(test_loader)
criterion = nn.CrossEntropyLoss()
quote_tensor = quote_tensor.to(device)


all_dfs = []
with torch.no_grad():
    # all_quote_ranks = []
    # all_quote_gs = []
    for i, (input_ids, token_type_ids, attention_masks, mask_ids, labels) in enumerate(test_loader):
        print(f"On data {i}/{len(test_loader)}")

        # decode what the test input is
        # 64 things in the batch
        print(f"input shape: {input_ids.shape}")
        # Decode the input_ids of sentence 0
        # this is decoding the sentence tokens --> words
        decoded_sentence = tokenizer.decode(input_ids[0], attention_mask=attention_masks[0])
        # print("input0:", decoded_sentence)

        input_ids = input_ids.to(device)

        token_type_ids = token_type_ids.to(device)
        attention_masks = attention_masks.to(device)
        mask_ids = mask_ids.to(device, dtype=torch.long)
        labels = labels.to(device, dtype=torch.long)
        outputs = model(input_ids, token_type_ids, attention_masks, mask_ids, quote_tensor)
        
        # print(f"outputs.shape: {outputs.shape}")
        # Print the list of NL sentences arranged by rank for input_ids[0]
        # print(f"labels: {labels}")
        # print(f"outputs for input0: {outputs}")
        # print("Predicted Sentences for input0:")
        
        # TODO: figure this out
        ps = outputs.data.cpu().numpy()
        # print(f"ps = {ps}")
        # print(f"len(ps): {len(ps)}")
        # in descending order
        sorted_indices = np.argsort(np.array(ps)*-1)
        # print(f"Sorted Indices: {sorted_indices}")
        # gs (gold standard?) represents the actual data number? of the quote in question
        gs = labels.cpu().numpy()
        # print(f"gs = {gs}")

        # cool, now I want the actual corresponding sentence.
        start = i*TEST_BATCH_SIZE
        end = min(len(y_test), start + len(labels))

        # create df from this part of data
        batch_df_map = {}

        batch_df_map["test_former"] = test_former[start:end]
        batch_df_map["test_latter"] = test_latter[start:end]
        batch_df_map["gs"] = gs
        # top k actual quotes
        top_k = 20
        for k in range(top_k):
            # kth most relevant result
            batch_df_map[f"res_{k+1}"] = [all_quotes[k_idx] for k_idx in sorted_indices[:,k]]
        
        # predicted ranks - list of lists
        batch_df_map["sorted_indices"] = sorted_indices.tolist()

        batch_df = pd.DataFrame(batch_df_map)

        # all_quote_ranks.append(sorted_indices, gs)
        all_dfs.append(batch_df)

final_df = pd.concat(all_dfs # , ignore_index=True
                     )

start test......
On data 0/200
input shape: torch.Size([64, 150])
On data 1/200
input shape: torch.Size([64, 150])
On data 2/200
input shape: torch.Size([64, 150])
On data 3/200
input shape: torch.Size([64, 150])
On data 4/200
input shape: torch.Size([64, 150])
On data 5/200
input shape: torch.Size([64, 150])
On data 6/200
input shape: torch.Size([64, 150])
On data 7/200
input shape: torch.Size([64, 150])
On data 8/200
input shape: torch.Size([64, 150])
On data 9/200
input shape: torch.Size([64, 150])
On data 10/200
input shape: torch.Size([64, 150])
On data 11/200
input shape: torch.Size([64, 150])
On data 12/200
input shape: torch.Size([64, 150])
On data 13/200
input shape: torch.Size([64, 150])
On data 14/200
input shape: torch.Size([64, 150])
On data 15/200
input shape: torch.Size([64, 150])
On data 16/200
input shape: torch.Size([64, 150])
On data 17/200
input shape: torch.Size([64, 150])
On data 18/200
input shape: torch.Size([64, 150])
On data 19/200
input shape: torch.Size([64,

In [219]:
final_df

,test_former,test_latter,gs,res_1,res_2,res_3,res_4,res_5,res_6,res_7,...,res_12,res_13,res_14,res_15,res_16,res_17,res_18,res_19,res_20,sorted_indices
0,archived video of that message here: biblical ...,which shew the work of the law written in the...,76,a law unto themselves.,a stiff-necked people.,"a government of laws, and not of men.",to keep law and order.,"the world, the flesh, and the devil.",from the strife of tongues.,tossing their heads in sprightly dance.,...,"tattlers also and busybodies, speaking things ...",let well alone.,the cattle upon a thousand hills.,treated like dirt.,by evil report and good report.,the defenders of democracy.,the palpable obscure.,the right divine of kings to govern wrong.,words of truth and soberness.,"[76, 170, 60, 5314, 4892, 1625, 5359, 5610, 10..."
1,"in the memories of his people, in the rest of ...","""blessed are the merciful! for they shall obt...",867,"blessed are the peacemakers, for they shall be...",blessed are they that mourn for they shall be ...,blessed are the merciful: for they shall obtai...,"blessed are the merciful, for they will be sho...",blessed are they which do hunger and thirst af...,blessed are the meek: for they shall inherit t...,"blessed are the meek, for they will inherit th...",...,"the sons of god saw the daughters of men, that...",blessed are ye that hunger now: for ye shall b...,"blessed are the young, for they shall inherit ...","and the angel said unto them, fear not: for, b...","blessed are the undefiled in the way, who walk...","but jesus turning unto them said, daughters of...","and god blessed noah and his sons, and said un...","and the lord commended the unjust steward, bec...",blessings brighten as they take their flight.,"[870, 875, 869, 868, 876, 867, 866, 5109, 1964..."
2,the human species depends on maximizing the po...,but we do no even have to be certain of this....,2784,injustice anywhere is a threat to justice ever...,there may be times when we are powerless to pr...,"the arc of the moral universe is long, but it ...",true peace is not merely the absence of tensio...,two possibilities exist: either we are alone i...,our lives begin to end the day we become silen...,rightful liberty is unobstructed action accord...,...,never in the field of human conflict was so mu...,is designed to make lies sound truthful and mu...,revenge is a kind of wild justice.,we cannot solve our problems with the same thi...,all we are saying is give peace a chance.,"it is better to be violent, if there is violen...",it is our duty to fight for our freedom.,in our age there is no such thing as 'keeping ...,there is no fate but what we make.,"[2784, 5047, 4337, 5379, 5413, 3788, 3991, 255..."
3,"said that he was 'carnal, sold under sin' . no...",this is my experience. i do many things i oug...,1545,"vengeance is mine, and i will repay.","vengeance is mine, i will repay, saith the lord.",vengeance is mine; i will repay.,i have sinned in that i have betrayed the inno...,for i acknowledge my transgressions: and my si...,"i find then a law, that, when i would do good,...",my punishment is greater than i can bear.,...,you cannot serve god and mammon.,he that hateth me hateth my father also.,he that loveth father or mother more than me i...,"ye have not yet resisted unto blood, striving ...",i make peace and create evil.,"i have not come to call the righteous, but sin...",i can do all things through christ which stren...,thou shalt not commit adultery.,"avenge not yourselves, but rather give place u...","[5442, 5443, 5444, 2296, 1510, 2241, 3425, 151..."
4,a female student run up to a man wearing a sik...,"over the exhortations of the speaker, the vid...",320,leave my mother alone!,"give me your tired, your poor, your huddled ma...",take my wife - please!,"give me liberty, or give me death.",god save us all.,"thank god, i have done my duty.","god, who am i?",...,you have to be with you.,three's a crowd.,save us from our friends.,it is our duty to fight for our freedom.

In [ ]:
final_df_small = final_df[sorted_indices]

In [228]:
# output the training data too
quotes_mapping = pd.DataFrame({"AllQuotes":all_quotes, "gs":np.arange(len(all_quotes))})
quotes_mapping

,AllQuotes,gs
0,'nothing tastes as good as skinny feels.',0
1,"'please, sir, i want some more.'",1
2,a babe in a house is a well-spring of pleasure.,2
3,a beggarly account of empty boxes.,3
4,a blonde to make a bishop kick a hole in a sta...,4
...,...,...
6103,"your task is not to seek for love, but merely ...",6103
6104,"your time is limited, so don't waste it living...",6104
6105,your word is your bond.,6105
6106,your work is going to fill a large part of you...,6106


In [208]:
# write out to excel
# with pd.ExcelWriter("QuoteR_output.xlsx") as writer:
#     # write each df to separate sheet
#     final_df.to_excel(writer, sheet_name="all_results", index=False),
#     test_data.to_excel(writer, sheet_name="test_data", index=False),
#     quotes_mapping.to_excel(writer, sheet_name="quotes_mapping", index=False)

In [229]:
# write out to parquet (more space efficient)
quote_retrievals_fname= "quote_retrievals"
quotes_mapping_fname = "quotes_mapping"
out_dirname = "out"

final_df.to_parquet(f"{out_dirname}/{quote_retrievals_fname}.parquet", index=False)
quotes_mapping.to_parquet(f"{out_dirname}/{quotes_mapping_fname}.parquet", index=False)

In [231]:
test_data = pd.DataFrame({"test_former": test_former, "test_quote":test_quote, "test_latter": test_latter, "gs": final_df["gs"]})
test_data

,test_former,test_quote,test_latter,gs
0,archived video of that message here: biblical ...,a law unto themselves.,which shew the work of the law written in the...,76
1,"in the memories of his people, in the rest of ...",blessed are the meek: for they shall inherit t...,"""blessed are the merciful! for they shall obt...",867
2,the human species depends on maximizing the po...,injustice anywhere is a threat to justice ever...,but we do no even have to be certain of this....,2784
3,"said that he was 'carnal, sold under sin' . no...",for the good that i would i do not; but the ev...,this is my experience. i do many things i oug...,1545
4,a female student run up to a man wearing a sik...,all we are saying is give peace a chance.,"over the exhortations of the speaker, the vid...",320
...,...,...,...,...
30,"examine the language of the church charters, r...",no religious test shall ever be required as a ...,"under the united states."" the provision is bi...",3538
31,"soon enough that we were not catholics, and wh...",out of the frying pan into the fire.,"with vengeance."" ""i think we might pass as sp...",3802
32,"her debut."" martin talbot, of the official cha...","whatever people say i am, that's what i'm not.",source: official uk charts company sales tren...,5674
33,"her over, 'till she's as desp'rate to recover ...",he that complies against his will is of his ow...,"which he may adhere to, yet disown, for reaso...",1872


In [239]:
# truncate the files for sharing
final_df_trunc = final_df.head(500)
test_data_trunc = test_data.head(500)

In [241]:
sharing_dirname = "share"

# quotes for mapping
final_df_trunc.to_parquet(f"{sharing_dirname}/{quote_retrievals_fname}.parquet", index=False)
quotes_mapping.to_parquet(f"{sharing_dirname}/{quotes_mapping_fname}.parquet", index=False)
# test data
test_data_fname = "test_data"
test_data_trunc.to_parquet(f"{sharing_dirname}/{test_data_fname}.parquet", index=False)

Read in data